In [5]:
instructions = open('input1.txt', 'r').read()
floor = 0
hit_basement = False
first_basement = 0
for index, instruction in enumerate(instructions):
    if instruction == '(':
        floor += 1
    else:
        floor -= 1
    if floor < 0 and not hit_basement:
        first_basement = index+1
        hit_basement = True
print(f'Santa ends up on floor {floor}')
print(f'He first hits the basement on instruction {first_basement}')

Santa ends up on floor 74
He first hits the basement on instruction 1795


In [17]:
boxes = [[int(x) for x in line.split('x')] for line in open('input2.txt', 'r').readlines()]
paper_area = 2 * sum(x*y + x*z + y*z + 0.5*min(x*y, x*z, y*z) for x,y,z in boxes)
print(f'The elves need {paper_area} square feet of wrapping paper')

ribbon = sum(min(2*(x+y), 2*(x+z), 2*(y+z)) + x*y*z for x,y,z in boxes)
print(f'The elves need {ribbon} feet of ribbon')

The elves need 1606483.0 square feet of wrapping paper
The elves need 3842356 feet of ribbon


In [28]:
instructions = open('input3.txt', 'r').read()
visited = set()
doubles = set()

first_santa = (0,0)
normal_santa = (0,0)
robo_santa = (0,0)

first_visited = set()
second_visited = set()

first_visited.add(first_santa)
second_visited.add(robo_santa)

def move(coordinate, instruction):
    x, y = coordinate
    if instruction == '>':
        return (x+1, y)
    elif instruction == '<':
        return (x-1, y)
    elif instruction == 'v':
        return (x, y-1)
    elif instruction == '^':
        return (x, y+1)

for index, instruction in enumerate(instructions):
    first_santa = move(first_santa, instruction)
    first_visited.add(first_santa)
    
    if index % 2 == 0:
        normal_santa = move(normal_santa, instruction)
        second_visited.add(normal_santa)
    else:
        robo_santa = move(robo_santa, instruction)
        second_visited.add(robo_santa)

print(f'Santa has visited {len(first_visited)} houses')
print(f'Santa and Robo-Santa have visited {len(second_visited)} houses')
    

Santa has visited 2592 houses
Santa and Robo-Santa have visited 2360 houses


In [1]:
import hashlib

input = 'yzbqklnj'
counter = 0

five_digits = 1e1000
six_digits = 1e1000

while True:
    m = hashlib.md5()
    m.update(f'{input}{counter}'.encode('utf-8'))
    if m.hexdigest().startswith('00000'):
        five_digits = counter if counter < five_digits else five_digits
    if m.hexdigest().startswith('000000'):
        six_digits = counter if counter < six_digits else six_digits
        break
    counter += 1
print(f'The lowest integer to produce a hash starting with five zeros is {five_digits}')
print(f'The lowest integer to produce a hash starting with six zeros is {six_digits}')

The lowest integer to produce a hash starting with five zeros is 282749
The lowest integer to produce a hash starting with six zeros is 9962624


In [10]:
import re
strings = open('input5.txt', 'r').readlines()

forbidden = '(ab|cd|pq|xy)'
double = '(.)\\1'
vowel = '[aeiou].*[aeiou].*[aeiou]'

def is_nice(string):
    return re.search(forbidden, string) is None and re.search(double, string) is not None and re.search(vowel, string) is not None

print(f'There are {sum(1 if is_nice(string) else 0 for string in strings)} nice strings')

double_repetition = '(..).*\\1'
space_repetition = '(.).\\1'

def is_new_nice(string):
    return re.search(double_repetition, string) is not None and re.search(space_repetition, string) is not None

print(f'There are {sum(1 if is_new_nice(string) else 0 for string in strings)} nice strings by the new rules')

There are 255 nice strings
There are 55 nice strings by the new rules


In [47]:
import re
from shapely.geometry import Polygon, Point
from shapely import intersects, intersection, unary_union

instructions = open('input6.txt', 'r').readlines()
reader = '(turn on|turn off|toggle) ([0-9]+),([0-9]+) through ([0-9]+),([0-9]+)'
turned_on = None
strengths = {}

for index, instruction in enumerate(instructions):
    print(f'\rPerforming instruction {index+1} of {len(instructions)}', end='')
    match = re.search(reader, instruction)
    x1 = int(match.group(2))
    x2 = int(match.group(4))
    y1 = int(match.group(3))
    y2 = int(match.group(5))

    area = Polygon([(x1, y1), (x1, y2+1), (x2+1, y2+1), (x2+1, y1)])
    if intersects(area, turned_on):
        affected = area.difference(turned_on)
        intersected = intersection(area, turned_on)
        still_on = turned_on.difference(area)
    
        if match.group(1) == 'turn on':
            turned_on = unary_union([affected, intersected, still_on])
        elif match.group(1) == 'toggle':
            turned_on = unary_union([affected, still_on])
        elif match.group(1) == 'turn off':
            turned_on = still_on
    else:
        if match.group(1) == 'turn on' or match.group(1) == 'toggle':
            turned_on = unary_union([area, turned_on])

    new_strengths = {}
    for strength_area in strengths:
        strength = strengths[strength_area]

        if intersects(area, strength_area):
            affected = intersection(area, strength_area)
            unaffected = strength_area.difference(area)
            area = area.difference(strength_area)

            new_strengths[unaffected] = strength
            if match.group(1) == 'turn on':
                new_strengths[affected] = strength + 1
            elif match.group(1) == 'toggle':
                new_strengths[affected] = strength + 2
            elif match.group(1) == 'turn off' and strength > 1:
                new_strengths[affected] = strength - 1
        else:
            new_strengths[strength_area] = strength

    if area.area > 0:
        if match.group(1) == 'turn on':
            new_strengths[area] = 1
        elif match.group(1) == 'toggle':
            new_strengths[area] = 2

    strengths = {unary_union([area for area, area_strength in new_strengths.items() if area_strength == strength]): strength for strength in set(new_strengths.values())}
        

print(f'\nThere are {turned_on.area} lights turned on')
print(f'The total brightness is {sum(area.area * strength for area, strength in strengths.items())}')

Performing instruction 300 of 300
There are 400410.0 lights turned on
{<GEOMETRYCOLLECTION (POLYGON ((19 70, 22 70, 22 45, 19 45, 19 70)), POLYGON ...>: 1, <GEOMETRYCOLLECTION (POINT (426 899), POINT (426 992), POLYGON ((101 104, 10...>: 2, <GEOMETRYCOLLECTION (POINT (86 527), POINT (266 995), POINT (534 899), POINT...>: 3, <GEOMETRYCOLLECTION (POINT (126 973), POINT (384 995), POINT (426 995), POLY...>: 4, <GEOMETRYCOLLECTION (POINT (289 973), POINT (547 936), POINT (689 899), POLY...>: 5, <GEOMETRYCOLLECTION (POINT (414 973), POINT (717 899), POLYGON ((231 44, 258...>: 6, <GEOMETRYCOLLECTION (POINT (86 583), POINT (688 899), POLYGON ((258 48, 231 ...>: 7, <GEOMETRYCOLLECTION (POINT (547 899), POINT (728 899), POLYGON ((227 55, 227...>: 8, <GEOMETRYCOLLECTION (POINT (688 899), POINT (732 899), POINT (801 117), POLY...>: 9, <GEOMETRYCOLLECTION (POINT (86 614), POINT (86 637), POINT (558 899), POLYGO...>: 10, <GEOMETRYCOLLECTION (POINT (749 899), POINT (903 935), POLYGON ((201 152, 20..

In [72]:
import re
instructions = open('input7.txt', 'r').readlines()

gates = {}

single_input = '^(\w+) -> (\w+)$'
not_input = '^NOT (\w+) -> (\w+)$'
and_input = '^(\w+) AND (\w+) -> (\w+)$'
or_input = '^(\w+) OR (\w+) -> (\w+)$'
lshift_input = '^(\w+) LSHIFT (\w+) -> (\w+)$'
rshift_input = '^(\w+) RSHIFT (\w+) -> (\w+)$'

target_check = '-> (\w+)$'
non_overwritable_target = ''

def get_value(input):
    if input.isdigit():
        return int(input)
    elif input in gates:
        return gates[input]

    return None

def perform_instructions():
    while len(instructions) > 0 and 'a' not in gates:
        performed_indices = []
        for index, instruction in enumerate(instructions):
            if re.search(target_check, instruction).group(1) == non_overwritable_target:
                print(instruction)
                performed_indices.append(index)
                continue
            
            performed = False
            if (match := re.search(single_input, instruction)) is not None:
                if (value := get_value(match.group(1))) is not None:
                    gates[match.group(2)] = value
                    performed = True
                    
            elif (match := re.search(not_input, instruction)) is not None:
                if (value := get_value(match.group(1))) is not None:
                    gates[match.group(2)] = value ^ 65535
                    performed = True
    
            elif (match := re.search(and_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = value1 & value2
                    performed = True
    
            elif (match := re.search(or_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = value1 | value2
                    performed = True
    
            elif (match := re.search(lshift_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = (value1 << value2) & 65535
                    performed = True
    
            elif (match := re.search(rshift_input, instruction)) is not None:
                if (value1 := get_value(match.group(1))) is not None and (value2 := get_value(match.group(2))) is not None:
                    gates[match.group(3)] = (value1 >> value2) & 65535
                    performed = True
    
            if performed:
                performed_indices.append(index)
    
        for performed_index in sorted(performed_indices, reverse=True):
            del instructions[performed_index]

perform_instructions()
print(f'The signal on wire a is {gates["a"]}')

gates = {'b': gates['a']}
non_overwritable_target = 'b'
instructions = open('input7.txt', 'r').readlines()
perform_instructions()
print(f'The new signal on wire a is {gates["a"]}')

The signal on wire a is 956
14146 -> b

The new signal on wire a is 40149


In [10]:
input = open('input8.txt', 'r').readlines()
bytes = sum(len(line) for line in input)
characters = bytes
longer = bytes
for line in input:
    characters -= 2
    i = 0
    while i < len(line):
        if line[i:].startswith('\\\\'):
            characters -= 1
            i += 2
        elif line[i:].startswith('\\"'):
            characters -= 1
            i += 2
        elif line[i:].startswith('\\x'):
            characters -= 3
            i += 4
        else:
            i += 1
print(f'The answer to part one is {bytes - characters}')

for line in input:
    longer += 2
    for c in line:
        if c == '\\':
            longer += 1
        elif c == '"':
            longer += 1

print(f'The answer to part two is {longer - bytes}')

The answer to part one is 1342
The answer to part two is 2074


In [22]:
import re, itertools
locations = {}

for line in open('input9.txt', 'r').readlines():
    if (match := re.search('(\w+) to (\w+) = ([0-9]+)', line)):
        place_one = match.group(1)
        place_two = match.group(2)
        distance = int(match.group(3))

        if place_one not in locations:
            locations[place_one] = {}
        if place_two not in locations:
            locations[place_two] = {}

        locations[place_one][place_two] = distance
        locations[place_two][place_one] = distance

def find_distance(locations_list):
    distance = 0
    for i in range(1, len(locations_list)):
        distance += locations[locations_list[i-1]][locations_list[i]]
    return distance

shortest_path = min(find_distance(route) for route in itertools.permutations(locations.keys()))
longest_path = max(find_distance(route) for route in itertools.permutations(locations.keys()))
print(f'The shortest possible path is {shortest_path}')
print(f'The longest possible path is {longest_path}')

The shortest possible path is 117
The longest possible path is 909


In [27]:
start = '3113322113'

def look_and_say(string):
    result = ''
    previous_character = string[0]
    length = 1
    for i in range(1, len(string)):
        if string[i] == previous_character:
            length += 1
        else:
            result += str(length) + previous_character
            previous_character = string[i]
            length = 1
    result += str(length) + previous_character
    return result

built_string = start
for i in range(40):
    built_string = look_and_say(built_string)
print(f'The length of the string after 40 times is {len(built_string)}')

The length of the string after 40 times is 329356


In [10]:
password = 'hepxcrrq'

def increment(password_string):
    last = ord(password_string[-1])
    if last == 122:
        return increment(password_string[:-1]) + 'a'
    else:
        return password_string[:-1] + chr(last+1)

def check_rules(password):
    if 'i' in password or 'o' in password or 'l' in password:
        return False

    pairs = 0
    has_straight = False
    counter = 0
    while counter < len(password):
        try:
            if password[counter] == password[counter + 1]:
                pairs += 1
                counter += 1
    
                if ord(password[counter+1]) - ord(password[counter]) == 1 and ord(password[counter+2]) - ord(password[counter+1]) == 1:
                    has_straight = True
        except IndexError:
            pass

        counter += 1

    return pairs > 1 and has_straight

incrementing_password = password
while not check_rules(incrementing_password):
    incrementing_password = increment(incrementing_password)
print(f'The next password that follows all the rules is {incrementing_password}')
incrementing_password = increment(incrementing_password)
while not check_rules(incrementing_password):
    incrementing_password = increment(incrementing_password)
print(f'The next password is {incrementing_password}')

The next password that follows all the rules is hepxxyzz
The next password is heqaabcc


In [31]:
import re, json
input = open('input12.txt', 'r').read()
matches = re.findall('[-]{0,1}[0-9]+', input)
print(f'The sum of all numbers contained in the document is {sum(int(x) for x in matches)}')

# Remove all objects with red as a property
python = json.loads(input)

def remove_red(object, parent, parent_property):
    

remove_red(python, None, None)
redless = json.dumps(python)
print(redless)
print(f'The sum of all numbers without red is {sum(int(x) for x in re.findall("[-]{0,1}[0-9]+", redless))}')

The sum of all numbers contained in the document is 119433


RuntimeError: dictionary changed size during iteration